## Personalizando conjuntos de dados em fastai

In [ ]:
from fastai.gen_doc.nbdoc import *
from fastai.vision import *

Neste tutorial, vamos ver como criar subclasses personalizadas de [`ItemBase`](/core.html#ItemBase) ou [`ItemList`](/data_block.html#ItemList) mantendo tudo a biblioteca fastai tem para oferecer. Para permitir que as funções básicas para trabalhar de forma consistente em várias aplicações, os delegados da biblioteca fastai várias tarefas a um desses objetos específicos, e vamos ver aqui quais métodos você tem que implementar para ser capaz de ter tudo funcionar corretamente. Mas primeiro vamos dar um passo para trás para ver onde você vai usar o seu resultado final.

## Ligações com a API bloco de dados

A API bloco de dados funciona, permitindo que você escolha uma classe que é responsável para obter seus itens e outra classe que é cobrado com a obtenção de seus alvos. Combinadas em conjunto, criam uma [`Dataset`](https://pytorch.org/docs/stable/data.html#torch.utils.data.Dataset) pytorch que é então enrolada no interior de um [`DataLoader`](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader). O conjunto de treinamento, conjunto de validação e talvez conjunto de teste são, então, todos colocados em um [`DataBunch`](/basic_data.html#DataBunch).
A API bloco de dados permite que você misturar e combinar que classe suas entradas têm, que classe seus alvos têm, como fazer a divisão entre trem e conjunto de validação, em seguida, como criar o [`DataBunch`](/basic_data.html#DataBunch), mas se você tem um tipo muito específico de entrada / target, as classes fastai pode não ser suficiente para você. Este tutorial está lá para explicar o que é necessário para criar uma nova classe de itens e que métodos são importantes para implementar ou substituir.
Ele vai em duas fases: primeiro vamos nos concentrar no que você precisa para criar uma classe [`ItemBase`](/core.html#ItemBase) personalizado (que é o tipo de seus entradas / alvos), em seguida, sobre como criar o seu [`ItemList`](/data_block.html#ItemList) personalizado (que é basicamente um conjunto de [`ItemBase`](/core.html#ItemBase)) ao destacar que métodos são chamados pela biblioteca.

## Criando uma subclasse [`ItemBase`](/core.html#ItemBase) personalizado

A biblioteca fastai contém três tipos básicos de [`ItemBase`](/core.html#ItemBase) que você pode querer subclasse:
- [`Image`](/vision.image.html#Image) para aplicações de visão
- [`Text`](/text.data.html#Text) para aplicativos de texto
- [`TabularLine`](/tabular.data.html#TabularLine) para aplicações tabulares
Se você decidir criar sua própria classe item ou a subclasse uma das opções acima, aqui está o que você precisa para implementar:

### atributos básicos

Esses são os atributos mais importantes do seu costume [`ItemBase`](/core.html#ItemBase) precisa de como eles são usados ​​em todos os lugares na biblioteca fastai:
- `ItemBase.data` é a coisa que é passado para pytorch quando você quer criar um [`DataLoader`](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader). Isto é o que precisa ser alimentado para o seu modelo. Note-se que pode ser diferente a partir da representação de seu artigo, desde que você pode querer algo que é mais compreensível.
- `representação __str__`: se for o caso, isso é o que será exibido quando a biblioteca fastai tem que mostrar o seu item.
Se tomarmos o exemplo de um objeto O` [`MultiCategory`](/core.html#MultiCategory) `por exemplo:
- `o.data` é um tensor onde as tags são um hot codificado
- `str (O)` retorna as etiquetas separadas por;
Se você quiser codificar o aumento de dados da maneira deve ser aplicado ao seu costume `Item`, você deve escrever um` apply_tfms` método. Isto é o que será chamado se você aplicar um bloco [`transform`](/vision.transform.html#vision.transform) na API do bloco de dados.

### Exemplo: ImageTuple

Para cycleGANs, precisamos criar um tipo personalizado de itens desde que alimentar os tuplos modelo de imagens. Vejamos como codificar isso. A base é codificar o atributo [`data`](/vision.data.html#vision.data) que é o que vai ser dada ao modelo. Note-se que nós ainda manter o controle do objeto inicial (usuall em um atributo `obj`) para ser capaz de mostrar agradáveis ​​representações mais tarde. Aqui, o objecto é o tuplo de imagens e os dados seus tensores subjacentes normalizados entre -1 e 1.

In [ ]:
class ImageTuple(ItemBase):
    def __init__(self, img1, img2):
        self.img1,self.img2 = img1,img2
        self.obj,self.data = (img1,img2),[-1+2*img1.data,-1+2*img2.data]

Em seguida, queremos aplicar o aumento de dados para o nosso tupla de imagens. Isso é feito por escrever um método `apply_tfms` como vimos antes. Aqui nós passar essa chamada para as duas imagens subjacentes, em seguida, atualizar os dados.

In [ ]:
    def apply_tfms(self, tfms, **kwargs):
        self.img1 = self.img1.apply_tfms(tfms, **kwargs)
        self.img2 = self.img2.apply_tfms(tfms, **kwargs)
        self.data = [-1+2*self.img1.data,-1+2*self.img2.data]
        return self

Nós definimos um último método para empilhar as duas imagens ao lado do outro, que vamos usar mais tarde para uma show_batch` comportamento `show_results` personalizado` /.

In [ ]:
    def to_one(self): return Image(0.5+torch.cat(self.data,2)/2)

Isto é tudo que você precisa para criar o seu [`ItemBase`](/core.html#ItemBase) personalizado. Você não será capaz de usá-lo até que você colocá-lo dentro de sua [`ItemList`](/data_block.html#ItemList) costume, porém, assim que você deve continuar a ler a próxima seção.

## Criando uma subclasse [`ItemList`](/data_block.html#ItemList) personalizado

Esta é a principal classe que permite que você agrupe suas entradas ou seus alvos na API do bloco de dados. então você pode usar qualquer um dos métodos de separação ou de rotulagem antes de criar um [`DataBunch`](/basic_data.html#DataBunch). Para se certificar que tudo está funcionando corretamente, aqui está o que você precisa saber.

### variáveis ​​de classe

Se você está subclasse diretamente [`ItemList`](/data_block.html#ItemList) ou um dos fastai particulares, certifique-se de conhecer o conteúdo das três variáveis ​​seguintes como pode ser necessário ajustá-los:
- `_bunch` contém o nome da classe que será usado para criar um [`DataBunch`](/basic_data.html#DataBunch)
- `_processor` contém uma classe (ou uma lista de classes) de [`PreProcessor`](/data_block.html#PreProcessor) que será, então, usado como padrão para criar processador para este [`ItemList`](/data_block.html#ItemList)
- `_label_cls` contém a classe que será usado para criar os rótulos por padrão
`_Label_cls` é o primeiro a ser utilizado na API bloco de dados, em função da rotulagem. Se esta variável está definido para `none`, a classe rótulo será definido para [`CategoryList`](/data_block.html#CategoryList), [`MultiCategoryList`](/data_block.html#MultiCategoryList) ou [`FloatList`](/data_block.html#FloatList) dependendo do tipo do primeiro item. O padrão pode ser substituído pela passagem de um `label_cls` nas kwargs da função de rotulagem.
`_Processor` é o segundo a ser utilizado. Os processadores são chamados no final da rotulagem para aplicar algum tipo de função em seus artigos. O processador padrão das entradas pode ser substituída por uma passagem `processor` nas kwargs ao criar o [`ItemList`](/data_block.html#ItemList), o processador padrão dos alvos pode ser substituído por meio de um` processor` nas kwargs da função de marcação.
Processadores são úteis para pré-processamento de alguns dados, mas você também precisa colocar em seu estado qualquer variável que você deseja salvar para a chamada de `data.export ()` antes de criar um objeto [`Learner`](/basic_train.html#Learner) para inferência: o estado do isn [`ItemList`](/data_block.html#ItemList) 't salvou lá, apenas os seus processadores. Por exemplo `única razão de SegmentationProcessor` de existir é para salvar as classes de conjunto de dados, e durante a chamada processo, ele não faz nada além de definir o` classes` e `atributos c` ao seu conjunto de dados.
``` python
class SegmentationProcessor(PreProcessor):
    def __init__(self, ds:ItemList): self.classes = ds.classes
    def process(self, ds:ItemList):  ds.classes,ds.c = self.classes,len(self.classes)
```
`_Bunch` é a variável última classe usada no bloco de dados. Quando você digita o `databunch final ()`, a API do bloco de dados chama o método `_bunch.create` com o` _bunch` das entradas.

### Mantendo \ _ \ _ o init \ _ \ _ argumentos

Se você passar argumentos adicionais em seu `__init__` chamar que você salve no seu estado de [`ItemList`](/data_block.html#ItemList), temos que ter certeza que eles também são repassados ​​no método` new` como esta é usada para criar a sua formação e validação set, quando divisão. Para fazer isso, você só tem que adicionar seus nomes no `argumento copy_new` do seu [`ItemList`](/data_block.html#ItemList) costume, de preferência durante o` __init__`. Aqui vamos precisar de duas coleções de nomes de arquivos (para os dois tipos de imagens) para que certifique-se o segundo é copiado como este:
```python
def __init__(self, items, itemsB=None, **kwargs):
    super().__init__(items, **kwargs)
    self.itemsB = itemsB
    self.copy_new.append('itemsB')
```
Certifique-se de manter os kwargs como é, como eles contêm todo o material adicional que você pode passar para um [`ItemList`](/data_block.html#ItemList).

### métodos importantes

#### - pegue

O método mais importante que você tem que implementar é `get`: este permitirá que o seu costume [`ItemList`](/data_block.html#ItemList) para gerar um [`ItemBase`](/core.html#ItemBase) da coisa armazenado em sua array` itens'um. Por exemplo, um [`ImageList`](/vision.data.html#ImageList) tem o seguinte método `get`:
``` python
def get(self, i):
    fn = super().get(i)
    res = self.open(fn)
    self.sizes[i] = res.size
    return res
```
A primeira linha, basicamente, olha para `self.items[i]` (which is a filename). A segunda linha abre desde o `open`method é apenas
``` python
def open(self, fn): return open_image(fn)
```
A terceira linha é lá para [`ImagePoints`](/vision.image.html#ImagePoints) ou [`ImageBBox`](/vision.image.html#ImageBBox) alvos que requerem o tamanho do [`Image`](/vision.image.html#Image) entrada a ser criado. Note que se você está construindo uma classe de destino personalizado e você precisa do tamanho de uma imagem, você deve chamar `self.x.size [i]`.

In [ ]:
jekyll_note("""If you just want to customize the way an `Image` is opened, subclass `Image` and just change the
`open` method.""")

<div markdown="span" class="alert alert-info" role="alert"><i class="fa fa-info-circle"></i> <b>Note: </b>If you just want to customize the way an `Image` is opened, subclass `Image` and just change the
`open` method.</div>

#### - reconstruir

Este é o método que é chamado na `data.show_batch ()`, `learn.predict ()` `ou learn.show_results ()` para transformar um tensor pytorch de volta para um [`ItemBase`](/core.html#ItemBase). De certa forma, ele faz o oposto de chamar `ItemBase.data`. Ele deve ter um tensor `t` e devolver o mesmo tipo de coisa que o método` get`.
Em algumas situações ([`ImagePoints`](/vision.image.html#ImagePoints), [`ImageBBox`](/vision.image.html#ImageBBox) por exemplo) você precisa ter um olhar para a entrada correspondente para reconstruir o seu item. Neste caso, você deve ter um segundo argumento chamado `x` (não altere esse nome). Por exemplo, aqui, é a `método reconstruct` de [`PointsItemList`](/vision.data.html#PointsItemList):
```python
def reconstruct(self, t, x): return ImagePoints(FlowField(x.size, t), scale=False)
```

#### - analyze_pred

Este é o método que é chamado na `learn.predict ()` `ou learn.show_results ()` previsões para transformar em um tensor de saída adequado para `reconstruct`. Por exemplo, nós pode precisar de tomar o argumento máximo (para [`Category`](/core.html#Category)) ou as previsões superiores a um determinado limite (para [`MultiCategory`](/core.html#MultiCategory)). Ele deve ter um tensor, juntamente com kwargs opcionais e retornar um tensor.
Por exemplo, aqui, é a `método analyze_pred` de [`MultiCategoryList`](/data_block.html#MultiCategoryList):
```python
def analyze_pred(self, pred, thresh:float=0.5): return (pred >= thresh).float()
```
`Thresh` pode, então, ser passado como kwarg durante as chamadas para` learn.predict () `` ou learn.show_results () `.

### métodos mostram avançadas

Se você quiser usar métodos como um `data.show_batch ()` ou `learn.show_results ()` com um novo tipo de [`ItemBase`](/core.html#ItemBase) você precisará implementar dois outros métodos. Em ambos os casos, a função genérica vai agarrar os tensores de insumos, metas e previsões (se aplicável), reconstruir a [`ItemBase`](/core.html#ItemBase) correspondente (como visto antes), mas ele vai delegar à [`ItemList`](/data_block.html#ItemList) o caminho para exibir os resultados.
``` python
def show_xys(self, xs, ys, **kwargs)->None:

def show_xyzs(self, xs, ys, zs, **kwargs)->None:
```
Em ambos os casos `xs` e` ys` representam as entradas e as metas, no segundo caso `zs` representam as previsões. Eles são listas do mesmo comprimento que dependem do `argumento rows` você passou. Os kwargs são passados ​​de data.show_batch `()` / `learn.show_results ()`. Como exemplo, aqui está o código fonte desses métodos em [`ImageList`](/vision.data.html#ImageList):
``` python
def show_xys(self, xs, ys, figsize:Tuple[int,int]=(9,10), **kwargs):
    "Show the `xs` and `ys` on a figure of `figsize`. `kwargs` are passed to the show method."
    rows = int(math.sqrt(len(xs)))
    fig, axs = plt.subplots(rows,rows,figsize=figsize)
    for i, ax in enumerate(axs.flatten() if rows > 1 else [axs]):
        xs[i].show(ax=ax, y=ys[i], **kwargs)
    plt.tight_layout()
    
def show_xyzs(self, xs, ys, zs, figsize:Tuple[int,int]=None, **kwargs):
    """Show `xs` (inputs), `ys` (targets) and `zs` (predictions) on a figure of `figsize`. 
    `kwargs` are passed to the show method."""
    figsize = ifnone(figsize, (6,3*len(xs)))
    fig,axs = plt.subplots(len(xs), 2, figsize=figsize)
    fig.suptitle('Ground truth / Predictions', weight='bold', size=14)
    for i,(x,y,z) in enumerate(zip(xs,ys,zs)):
        x.show(ax=axs[i,0], y=y, **kwargs)
        x.show(ax=axs[i,1], y=z, **kwargs)
```
Ligado a este método é a variável de classe `_show_square` de um [`ItemList`](/data_block.html#ItemList). O padrão é `false` mas se é` true`, o método `show_batch` enviará` linhas * rows` `xs` e` ys` para `show_xys` (de modo que ele mostra um quadrado de entradas / alvos), como aqui para imagens.

### Exemplo: ImageTupleList

Continuando com nosso exemplo de item personalizado, criamos uma classe [`ItemList`](/data_block.html#ItemList) personalizado que vai embrulhar essas `ImageTuple`s corretamente. A primeira coisa é escrever um `costume __init__` método (uma vez que precisamos de uma lista de nomes aqui) o que significa que também temos que mudar o método` new`.

In [ ]:
class ImageTupleList(ImageList):
    def __init__(self, items, itemsB=None, **kwargs):
        super().__init__(items, **kwargs)
        self.itemsB = itemsB
        self.copy_new.append('itemsB')

Em seguida, especificar como obter um item. Aqui passamos a imagem na primeira lista de itens, e escolher um aleatoriamente na segunda lista.

In [ ]:
    def get(self, i):
        img1 = super().get(i)
        fn = self.itemsB[random.randint(0, len(self.itemsB)-1)]
        return ImageTuple(img1, open_image(fn))

Nós também adicionar um método de fábrica personalizada para criar diretamente uma `ImageTupleList` de duas pastas.

In [ ]:
    @classmethod
    def from_folders(cls, path, folderA, folderB, **kwargs):
        itemsB = ImageList.from_folder(path/folderB).items
        res = super().from_folder(path/folderA, itemsB=itemsB, **kwargs)
        res.path = path
        return res

Finalmente, temos de especificar como reconstruir o `ImageTuple` de tensores se quisermos` show_batch` para trabalhar. Nós recriar as imagens e desnormalizar.

In [ ]:
    def reconstruct(self, t:Tensor): 
        return ImageTuple(Image(t[0]/2+0.5),Image(t[1]/2+0.5))

Não há necessidade de escrever um método `analyze_preds` desde o comportamento padrão (retornando o tensor de saída) é o que precisamos aqui. No entanto `show_results` não funcionará corretamente a menos que o alvo (que realmente não se preocupam aqui) tem o método` reconstruct` direita: a biblioteca fastai usa o método `reconstruct` do alvo nas saídas. É por isso que criar outro [`ItemList`](/data_block.html#ItemList) personalizado com apenas que `método reconstruct`. A primeira linha é para reconstruir os nossos objectivos do manequim, e o segundo é o mesmo que no `ImageTupleList`.

In [ ]:
class TargetTupleList(ItemList):
    def reconstruct(self, t:Tensor): 
        if len(t.size()) == 0: return t
        return ImageTuple(Image(t[0]/2+0.5),Image(t[1]/2+0.5))

Para garantir que a `ImageTupleList` usa isso para rotulagem, nós passá-lo em` _label_cls` e é isso que o resultado parece.

In [ ]:
class ImageTupleList(ImageList):
    _label_cls=TargetTupleList
    def __init__(self, items, itemsB=None, **kwargs):
        super().__init__(items, **kwargs)
        self.itemsB = itemsB
        self.copy_new.append('itemsB')
    
    def get(self, i):
        img1 = super().get(i)
        fn = self.itemsB[random.randint(0, len(self.itemsB)-1)]
        return ImageTuple(img1, open_image(fn))
    
    def reconstruct(self, t:Tensor): 
        return ImageTuple(Image(t[0]/2+0.5),Image(t[1]/2+0.5))
    
    @classmethod
    def from_folders(cls, path, folderA, folderB, **kwargs):
        itemsB = ImageList.from_folder(path/folderB).items
        res = super().from_folder(path/folderA, itemsB=itemsB, **kwargs)
        res.path = path
        return res

Por último, queremos personalizar o comportamento do `show_batch` e` show_results`. Lembre-se o método `to_one` apenas coloca as duas imagens ao lado do outro.

In [ ]:
    def show_xys(self, xs, ys, figsize:Tuple[int,int]=(12,6), **kwargs):
        "Show the `xs` and `ys` on a figure of `figsize`. `kwargs` are passed to the show method."
        rows = int(math.sqrt(len(xs)))
        fig, axs = plt.subplots(rows,rows,figsize=figsize)
        for i, ax in enumerate(axs.flatten() if rows > 1 else [axs]):
            xs[i].to_one().show(ax=ax, **kwargs)
        plt.tight_layout()

    def show_xyzs(self, xs, ys, zs, figsize:Tuple[int,int]=None, **kwargs):
        """Show `xs` (inputs), `ys` (targets) and `zs` (predictions) on a figure of `figsize`.
        `kwargs` are passed to the show method."""
        figsize = ifnone(figsize, (12,3*len(xs)))
        fig,axs = plt.subplots(len(xs), 2, figsize=figsize)
        fig.suptitle('Ground truth / Predictions', weight='bold', size=14)
        for i,(x,z) in enumerate(zip(xs,zs)):
            x.to_one().show(ax=axs[i,0], **kwargs)
            z.to_one().show(ax=axs[i,1], **kwargs)